In [162]:
from osgeo import gdal
from osgeo import ogr
import pandas as pd
import struct
import math
from scipy import interpolate
import numpy as np

In [32]:
def pt2fmt(pt):
    fmttypes = {
        gdal.GDT_Byte: 'B',
        gdal.GDT_Int16: 'h',
        gdal.GDT_UInt16: 'H',
        gdal.GDT_Int32: 'i',
        gdal.GDT_UInt32: 'I',
        gdal.GDT_Float32: 'f',
        gdal.GDT_Float64: 'f'
    }
    return fmttypes.get(pt, 'x')

In [127]:
def read_exactly(fd, size):
    data = b""
    remaining = size
    while remaining:
        newdata = fd.read(remaining)
        if len(newdata)==0:
            raise IOError("Failed to read enough data")
        data += newdata
        remaining -= len(newdata)
    return data

In [235]:
def bilinear_interpolate_grid(z, x, y):
    x = x % 1
    y = y % 1
    return z[0][0] * (1-x) * (1-y) + z[1][0] * x * (1-y) + z[0][1] * (1-x) * y + z[1][1] * x * y
    

In [237]:
# Coords.
lat = 37.324529
lon = -122.170777

In [232]:
12.04 % 1

0.03999999999999915

In [238]:
# TIF file.
tif_path = './data/ETOPO1/ETOPO1_Ice_g_geotiff.tif'
dataset = gdal.Open(tif_path, gdal.GA_ReadOnly)
band = dataset.GetRasterBand(1)

gt = dataset.GetGeoTransform() 
gt_inv = gdal.InvGeoTransform(gt)

grid_x, grid_y = gdal.ApplyGeoTransform(gt_inv, lon, lat)
structval = band.ReadRaster(int(grid_x), int(grid_y), 1, 1, buf_type = band.DataType)
fmt = pt2fmt(band.DataType)
intval = struct.unpack(fmt , structval)
elevation = intval[0]

print(grid_x)
print(grid_y)
print(elevation)

# Interpolate.
w = math.floor(grid_x)
e = math.ceil(grid_x)
n = math.floor(grid_y)
s = math.ceil(grid_y)
x = [w, e, e, w]
y = [n, n, s, s]
elevations = [band.ReadRaster(xx, yy, 1, 1, buf_type = band.DataType) for xx, yy in zip(x, y)]
elevations = [struct.unpack(fmt , s)[0] for s in elevations]
x = [w, e]
y = [n, s]
elevations = [[elevations[0], elevations[1]], [elevations[3], elevations[2]]]
elevation_interp = interpolate.interpn((x, y), elevations, (grid_x, grid_y), method='linear')
print()
print(elevations)
print(elevation_interp)
print(bilinear_interpolate_grid(elevations, grid_x, grid_y))

3470.25338
3161.0282600000014
641

[[641, 738], [648, 562]]
[ 644.20450506]
644.2045050596715


In [227]:
RGI = interpolate.RegularGridInterpolator

x = np.linspace(0, 1, 2) #  or  0.5*np.arange(3.) works too

# populate the 3D array of values (re-using x because lazy)
X, Y = np.meshgrid(x, x, indexing='ij')
vals = np.sin(X) + np.cos(Y)

# make the interpolator, (list of 1D axes, values at all points)
rgi = interpolate.RegularGridInterpolator(points=[x, x], values=vals, method='linear')  # can also be [x]*3 or (x,)*3

tst = (0.47, 0.49)

print(rgi(tst))
print(np.sin(tst[0]) + np.cos(tst[1]))

1.1702394927350999
1.33521914399


In [228]:
vals.tolist()

[[1.0, 0.5403023058681398], [1.8414709848078965, 1.3817732906760363]]

In [213]:
x = [.5, 2., 3., 4., 5.5]
y = [.5, 2., 3., 4., 5.5]
z = [[1, 2, 1, 2, 1], [1, 2, 1, 2, 1], [1, 2, 3, 2, 1],[1, 2, 2, 2, 1], [1, 2, 1, 2, 1]]
xi = np.array([[1, 2.3, 5.3, 0.5, 3.3, 1.2, 3],[1, 3.3, 1.2, 4.0, 5.0, 1.0, 3]]).T

# interpolate.interpn((x, y), z, xi, method='linear')
interpolate.interpn((x, y), z, xi, method='linear')


array([ 1.33333333,  1.72      ,  1.46666667,  2.        ,  1.33333333,
        1.33333333,  3.        ])

In [142]:
# Binary file.
n_cols = 21601
n_rows = 10801
cell_size = 1/60
nodata_value = -32768
bin_path = './data/ETOPO1/etopo1_ice_g_i2.bin'
bytes_per_cell = 2

# Coords
grid_x = (lon + 180 + cell_size/2)/(360 + cell_size) * n_cols
grid_y = (90 - lat+ cell_size/2)/(180 + cell_size) * n_rows
with open(bin_path, 'rb') as f:
    n_skip_cells = int(grid_y) * n_cols + int(grid_x)
    n_skip_bytes = n_skip_cells * bytes_per_cell
    f.seek(n_skip_bytes, 0)
    elevation = read_exactly(f, bytes_per_cell)
    elevation = struct.unpack('h', elevation)[0]
    
print(grid_x)
print(grid_y)
print(elevation)
    
# Multi coords.
w = math.floor(grid_x)
e = math.ceil(grid_x)
n = math.floor(grid_y)
s = math.ceil(grid_y)
x = [w, e, e, w]
y = [n, n, s, s]
elevations = []

with open(bin_path, 'rb') as f:
    n_skip_cells = int(n) * n_cols + int(w)
    n_skip_bytes = n_skip_cells * bytes_per_cell
    f.seek(n_skip_bytes, 0)
    elevation = read_exactly(f, bytes_per_cell)
    elevation = struct.unpack('h', elevation)[0]
    elevations.append(elevation)
    
    n_skip_cells = int(n) * n_cols + int(e)
    n_skip_bytes = n_skip_cells * bytes_per_cell
    f.seek(n_skip_bytes, 0)
    elevation = read_exactly(f, bytes_per_cell)
    elevation = struct.unpack('h', elevation)[0]
    elevations.append(elevation)
    
    n_skip_cells = int(s) * n_cols + int(w)
    n_skip_bytes = n_skip_cells * bytes_per_cell
    f.seek(n_skip_bytes, 0)
    elevation = read_exactly(f, bytes_per_cell)
    elevation = struct.unpack('h', elevation)[0]
    elevations.append(elevation)
    
    n_skip_cells = int(s) * n_cols + int(e)
    n_skip_bytes = n_skip_cells * bytes_per_cell
    f.seek(n_skip_bytes, 0)
    elevation = read_exactly(f, bytes_per_cell)
    elevation = struct.unpack('h', elevation)[0]
    elevations.append(elevation)
    

print(elevations)

3470.2533799999997
3161.0282599999996
641
[641, 738, 648, 562]


In [140]:
s

3162

In [92]:
# XLLCENTER = -180.000000
# YLLCENTER = -90.000000
# CELLSIZE = 0.01666666667
# BYTEORDER = LSBFIRST
# NUMBERTYPE = 2_BYTE_INTEGER
# ZUNITS = METERS
# MIN_VALUE = -10898
# MAX_VALUE = 8271

0.016666666666666666